In [1]:
import json
import requests
import config
import re

def getRequest(url, username, password, parameters = {}, debug = False): 
    r = requests.get(url, params=parameters, auth=(username, password))
    if debug:
        print('Status Code: %i'%r.status_code)
    return r


def getRequestJson(url, username, password, parameters = {}, debug = False, drill = False): 
    print(url)
    r = requests.get(url, params=parameters, auth=(username, password))
    if debug:
        print('Status Code: %i'%r.status_code)
    if not drill:
        return r.json()
    else:
        page_count = findPageCount(r.links)
        if page_count == 1:
            return r.json()
        current_page = findCurrentPage(url)
        if debug:
            print('page: %i of %i'%(current_page, page_count))
        if page_count == current_page:
            return r.json()
        else:
            return r.json() + getRequestJson(r.links['next']['url'], username, password, parameters, debug, drill)
        
def headRequest(url, username, password, parameters = {}, debug = False): 
    r = requests.head(url, params=parameters, auth=(username, password))
    if degug:
        print('Status Code: %i'%r.status_code)
    return r

def findPageCount(links, search = 'page=([^,]+)'):
    if not links:
        return 1
    try:
        match = re.search(search, links['last']['url'])
        if match:
            count = int(match.group(1))
            return count
        else:
            return False
    except KeyError:
        print('should be the last one' + json.dumps(links))
        match = re.search(search, links['prev']['url'])
        if match:
            count = int(match.group(1)) + 1
            return count
        else:
            return False

def findCurrentPage(url, search = 'page=([^,]+)'):
    if 'page=' in url:
        match = re.search(search, url)
        if match:
            count = int(match.group(1))
            return count
        else:
            return False
    else:
        return 1
    
def formMilestoneByRepoUrl(base, owner, repo):
    return('%s/repos/%s/%s/milestones'%(base, owner, repo))

In [2]:
teams = getRequestJson(config.URL_TEAMS, config.USER, config.TOKEN, drill = True)
for team in teams:
    print(team['name'])

https://api.github.com/orgs/Beaver-Design/teams
Construction
Debris Collection
Engineering
Environmental Monitoring


In [3]:
repos = getRequestJson(config.URL_REPOS, config.USER, config.TOKEN, drill = True)
for repo in repos:
    print(repo['name'])

https://api.github.com/orgs/Beaver-Design/repos
github-organizer


In [4]:
milestones = []
for repo in repos:
    url = formMilestoneByRepoUrl(config.URL_BASE, config.ORG, repo['name'])
    r = getRequest(url, config.USER, config.TOKEN)
    milestones+=r.json()

In [5]:
for milestone in milestones:
    print(milestone['title'])

get'r done


In [6]:
parameters = {'filter':'all'}
issues = getRequestJson(config.URL_ISSUES, config.USER, config.TOKEN, parameters, drill = True)
for index, issue in enumerate(issues):
    print(index, issue['title'])
    

https://api.github.com/orgs/Beaver-Design/issues
0 Needs more milestones
1 Needs more colors
2 Needs more levels


In [7]:
url = r'https://api.github.com/search/issues?q=user:Beaver-Design+state:open'
issue_count = getRequest(url,  config.USER, config.TOKEN)
total_count = issue_count.json()['total_count']
print('Total open issue count: %i'%total_count)

Total open issue count: 3
